In [1]:
import pandas as pd
import requests
from datetime import datetime
import time
import yfinance as yf

In [2]:
headers = {'User-Agent': "kotavenkateshgnanashri@gmail.com"}

In [3]:
company_tickers_all = requests.get("https://www.sec.gov/files/company_tickers.json", headers = headers)

In [4]:
first_ticker_preview = company_tickers_all.json()['0']
print(first_ticker_preview)

{'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'}


In [5]:
cik_val_check = company_tickers_all.json()['0']['cik_str']
print(cik_val_check)


1045810


In [6]:
company_data = pd.DataFrame.from_dict(company_tickers_all.json(), orient='index')
company_data[:1]

,cik_str,ticker,title
0,1045810,NVDA,NVIDIA CORP


In [7]:
company_data['cik_str'] = company_data['cik_str'].astype(str).str.zfill(10)
company_data[:10]

,cik_str,ticker,title
0,0001045810,NVDA,NVIDIA CORP
1,0000320193,AAPL,Apple Inc.
2,0000789019,MSFT,MICROSOFT CORP
3,0001652044,GOOGL,Alphabet Inc.
4,0001018724,AMZN,AMAZON COM INC
5,0001730168,AVGO,Broadcom Inc.
6,0001326801,META,"Meta Platforms, Inc."
7,0001318605,TSLA,"Tesla, Inc."
8,0001067983,BRK-B,BERKSHIRE HATHAWAY INC
9,0000059478,LLY,ELI LILLY & Co


In [8]:
len(company_data)

10196

In [9]:
company_data = company_data.head(200)
len(company_data)

200

In [10]:
cik = company_data['cik_str'].iloc[0]
print(cik)

0001045810


In [11]:

submission_dara_url =  requests.get(f'https://data.sec.gov/submissions/CIK{cik}.json', headers=headers)


In [12]:
print(submission_dara_url.json().keys())

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'ownerOrg', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'lei', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])


In [13]:
print(submission_dara_url.json()['cik'])
entityType = submission_dara_url.json()['entityType']
print(entityType)
print(submission_dara_url.json()['sicDescription'])
print(submission_dara_url.json()['name'])
print(submission_dara_url.json()['addresses']['business']["zipCode"])

0001045810
operating
Semiconductors & Related Devices
NVIDIA CORP
95051


In [14]:
company_facts = requests.get(
    f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
    headers=headers
    )

In [15]:
# print(company_facts.json().keys())
# print(company_facts.json()['facts'].keys())
# print(company_facts.json()['facts']['us-gaap'].keys())
# print(company_facts.json()['facts']['us-gaap'].keys())
# print(company_facts.json()['facts']['us-gaap']['Revenues'].keys())
# print(company_facts.json()['facts']['us-gaap']['Revenues']["label"])
# print(company_facts.json()['facts']['us-gaap']['Revenues']["units"]["USD"])
# revenues = company_facts.json()['facts']['us-gaap']['Revenues']["units"]["USD"]
# vals = [entry['val'] for entry in revenues]
# print(vals)

In [16]:
from collections import defaultdict

revenues = company_facts.json()['facts']['us-gaap']['Revenues']["units"]["USD"]

current_year = datetime.now().year

def frame_year(frame):
    if frame and frame.startswith("CY"):
        return int(frame[2:6])
    return None

calendar_revenue = {}

for entry in revenues:
    if 'frame' in entry:
        year = frame_year(entry['frame'])
        if year and current_year - 5 < year <= current_year:
            if entry['frame'] == f"CY{year}":
                calendar_revenue[year] = entry['val']

quarterly_sums = defaultdict(int)
for entry in revenues:
    if 'frame' in entry:
        year = frame_year(entry['frame'])
        if year and current_year - 5 < year <= current_year:
            if year not in calendar_revenue and entry['frame'].startswith(f"CY{year}Q"):
                quarterly_sums[year] += entry['val']

for year, val in quarterly_sums.items():
    calendar_revenue[year] = val

for year in sorted(calendar_revenue):
    print(f"{year}: {calendar_revenue[year]:,} USD")


2021: 26,914,000,000 USD
2022: 26,974,000,000 USD
2023: 60,922,000,000 USD
2024: 130,497,000,000 USD
2025: 147,811,000,000 USD


In [17]:

resp = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
sec_data = resp.json()

tickers_df = pd.DataFrame.from_dict(sec_data, orient='index')
tickers_df.columns = ["cik", "ticker", "company_name"]

company_list = tickers_df["ticker"].head(400).tolist()

print(company_list)



['NVDA', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'AVGO', 'META', 'TSLA', 'BRK-B', 'LLY', 'WMT', 'JPM', 'V', 'ORCL', 'SPY', 'XOM', 'JNJ', 'MA', 'NFLX', 'ABBV', 'COST', 'ASML', 'BAC', 'PLTR', 'BABA', 'PG', 'AMD', 'HD', 'GE', 'KO', 'CVX', 'CSCO', 'UNH', 'CYATY', 'AZN', 'SAP', 'IBM', 'WFC', 'CAT', 'MS', 'TM', 'PM', 'AXP', 'NVS', 'MRK', 'TMUS', 'GS', 'HSBC', 'RTX', 'QQQ', 'MU', 'MCD', 'ABT', 'CRM', 'TMO', 'NVO', 'SHEL', 'RY', 'PEP', 'ISRG', 'LIN', 'HDB', 'SHOP', 'DIS', 'AMGN', 'T', 'C', 'INTU', 'APP', 'LRCX', 'NEE', 'AMAT', 'UBER', 'QCOM', 'VZ', 'MUFG', 'BX', 'NOW', 'SONY', 'TJX', 'SCHW', 'INTC', 'PDD', 'APH', 'DHR', 'DTEGY', 'GILD', 'BLK', 'GEV', 'ANET', 'ACN', 'SPGI', 'SAN', 'BKNG', 'UL', 'KLAC', 'BSX', 'ARM', 'TD', 'TXN', 'PFE', 'SYK', 'TTE', 'HTHIY', 'BA', 'RTNTF', 'WELL', 'BHP', 'PGR', 'ADBE', 'UNP', 'UTX', 'COF', 'DE', 'LOW', 'ETN', 'MDT', 'HON', 'PANW', 'CRWD', 'SPOT', 'SNY', 'BTI', 'BUD', 'CB', 'BBVA', 'UBS', 'PLD', 'RIO', 'IBN', 'HCA', 'ADI', 'LMT', 'COP', 'VRTX', 'CEG', 'SMFG', 'KKR', 'E

In [18]:
stock = yf.Ticker("TRGP")
fin = stock.income_stmt

print(fin.index)
print(stock.balance_sheet.index)
print(stock.cashflow.index)

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Total Operating Income As Reported',
       'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS',
       'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Otherunder Preferred Stock Dividend',
       'Preferred Stock Dividends', 'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Income Expense', 'Other Non Operating Income Expenses',
     

In [19]:
def get_company_info(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        company_name = info.get("longName")
        industry = info.get("industry")
        country = info.get("country")
        currency = info.get("financialCurrency") or info.get("currency")

        return company_name, industry, country, currency

    except Exception as e:
        print(f"Error in company info for {ticker}: {e}")
        return None, None, None, None


In [79]:
def get_financials(ticker):
    stock = yf.Ticker(ticker)

    def is_valid_revenue(val):
        """Reject corrupted Yahoo values."""
        if val is None:
            return False
        if pd.isna(val):
            return False
        if val <= 0:               # Revenue cannot be zero or negative
            return False
        if val < 1e5:              # Anything under 100k is most probably fake 
            return False
        return True

    revenues = {}

    try:
        inc = stock.income_stmt
        if "Total Revenue" in inc.index:
            for date, value in inc.loc["Total Revenue"].items():
                year = date.year
                if is_valid_revenue(value):
                    revenues[year] = value
    except:
        pass

    try:
        q_inc = stock.quarterly_income_stmt
        if "Total Revenue" in q_inc.index:
            q_revenues = {}

            for date, value in q_inc.loc["Total Revenue"].items():
                year = date.year
                q_revenues.setdefault(year, 0)
                if not pd.isna(value):
                    q_revenues[year] += value   

            for yr, total in q_revenues.items():
                if is_valid_revenue(total):
                    revenues.setdefault(yr, total)
    except:
        pass

    try:
        val = stock.fast_info.get("last_total_revenue")
        if is_valid_revenue(val):
            latest_year = max(revenues.keys()) + 1 if revenues else datetime.now().year
            revenues.setdefault(latest_year, val)
    except:
        pass

    if not revenues:
        return []  # nothing valid

    clean_rows = []
    for yr, revenue in sorted(revenues.items(), reverse=True)[:3]:
        clean_rows.append({
            "year": yr,
            "revenue": revenue
        })

    return clean_rows


KPIs to show : https://www.kippy.cloud/post/kpis-to-effectively-measure-business-performance

Return on Assets (ROA) = Net Income ÷ Average Total Assets


https://www.wallstreetprep.com/knowledge/return-on-assets-roa/

In [80]:
def get_roa(ticker):
    try:
        stock = yf.Ticker(ticker)

        income = stock.income_stmt
        bs = stock.balance_sheet
        if "Net Income" not in income.index or "Total Assets" not in bs.index:
            return None
        net_income = income.loc["Net Income"]
        total_assets = bs.loc["Total Assets"]

        # index to years
        ni_years = {d.year: v for d, v in net_income.items()}
        ta_years = {d.year: v for d, v in total_assets.items()}

        roa = {}
        for yr in ni_years:
            if yr in ta_years and ta_years[yr] != 0:
                roa[yr] = ni_years[yr] / ta_years[yr]

        return roa if roa else None

    except Exception as e:
        print(f"Error getting ROA for {ticker}: {str(e)}")
        return None


Net Profit Margin
https://www.investopedia.com/terms/n/net_margin.asp

Net Profit Margin = (Net Profit / Total Revenue) * 100

In [81]:
def get_net_profit_margin(ticker):
    try:
        stock = yf.Ticker(ticker)
        inc = stock.income_stmt
        if "Net Income" not in inc.index or "Total Revenue" not in inc.index:
            return {}
        net_income = inc.loc["Net Income"]
        total_revenue = inc.loc["Total Revenue"]

        npm = {}

        for date in net_income.index:
            if date in total_revenue.index and total_revenue[date] != 0:
                npm[date.year] = (net_income[date] / total_revenue[date]) * 100

        return npm

    except Exception as e:
        print("Net Profit Margin error for", ticker, e)
        return {}


https://www.xero.com/guides/what-is-gross-profit-margin/

Gross profit margin = (Gross profit / Total revenue) * 100

In [82]:
def get_gross_profit_margin(ticker):
    try:
        stock = yf.Ticker(ticker)
        inc = stock.income_stmt
        if "Gross Profit" not in inc.index or "Total Revenue" not in inc.index:
            return {}
        gross_profit = inc.loc["Gross Profit"]
        total_revenue = inc.loc["Total Revenue"]

        gpm = {}

        for date in gross_profit.index:
            if date in total_revenue.index and total_revenue[date] != 0:
                gpm[date.year] = (gross_profit[date] / total_revenue[date]) * 100

        return gpm

    except Exception as e:
        print("Gross Profit Margin error for", ticker, e)
        return {}


EPS : Earnings Per Share
Can get from yfinance directly

In [83]:
def get_eps(ticker):
    try:
        inc = yf.Ticker(ticker).income_stmt
        basic = inc.loc["Basic EPS"] if "Basic EPS" in inc.index else None
        diluted = inc.loc["Diluted EPS"] if "Diluted EPS" in inc.index else None
        
        if basic is None and diluted is None:
            return {}
        
        dates = basic.index if basic is not None else diluted.index
        return {
            date.year: {
                "basic_eps": basic[date] if basic is not None else None,
                "diluted_eps": diluted[date] if diluted is not None else None
            }
            for date in dates
        }
    except:
        return {}

ROE = Net Income / Shareholders’ Equity
​
https://www.investopedia.com/terms/r/returnonequity.asp

In [84]:
def get_roe(ticker):
    try:
        stock = yf.Ticker(ticker)
        inc = stock.income_stmt
        bs = stock.balance_sheet
        if "Net Income" not in inc.index:
            return None
        
        equity_fields = ["Stockholders Equity", "Common Stock Equity"]
        equity_row = None
        
        for field in equity_fields:
            if field in bs.index:
                equity_row = bs.loc[field]
                break

        if equity_row is None:
            return None
        
        net_income = inc.loc["Net Income"]

        roe = {}

        for date in net_income.index:
            if date in equity_row.index:
                if equity_row.loc[date] != 0:
                    roe[date] = net_income.loc[date] / equity_row.loc[date]

        return roe if roe else None

    except Exception as e:
        print(f"Error calculating ROE for {ticker}: {e}")
        return None


In [ ]:
def process_company(ticker):
    name, industry, country , currency= get_company_info(ticker)
    revenues = get_financials(ticker)
    dict_roa = get_roa(ticker)
    dict_npm = get_net_profit_margin(ticker)
    dict_gpm = get_gross_profit_margin(ticker)
    dict_eps = get_eps(ticker)
    dict_roe = get_roe(ticker)

    rows = []
    for r in revenues:
        year = r["year"]
        eps_info = dict_eps.get(year, {})
        roe_value = None
        if dict_roe:
            for d, v in dict_roe.items():
                if d.year == r["year"]:
                    roe_value = v
                    break

        rows.append({
            "ticker": ticker,
            "company_name": name,
            "country": country,
            "industry": industry,
            "year": r["year"],
            "revenue": r["revenue"],
            "revenue_unit": currency,
            "ROA": dict_roa.get(year),
            "net_profit_margin": dict_npm.get(year),
            "gross_profit_margin": dict_gpm.get(year),
            "basic_eps": eps_info.get("basic_eps"),
            "diluted_eps": eps_info.get("diluted_eps"),
            "roe": roe_value
        })

    return rows

In [86]:
all_rows = []
for t in company_list:
    rows = process_company(t)
    all_rows.extend(rows)
    time.sleep(0.3)

df = pd.DataFrame(all_rows)


In [88]:
raw_data_frame = df

In [133]:
raw_data_frame

,ticker,company_name,country,industry,year,revenue,revenue_unit,ROA,net_profit_margin,gross_profit_margin,basic_eps,diluted_eps,roe
0,NVDA,NVIDIA Corporation,United States,Semiconductors,2025,1.304970e+11,USD,0.653041,55.848027,74.988697,2.970000,2.940000,0.918729
1,NVDA,NVIDIA Corporation,United States,Semiconductors,2024,6.092200e+10,USD,0.452775,48.849348,72.717573,1.210000,1.190000,0.692447
2,NVDA,NVIDIA Corporation,United States,Semiconductors,2023,2.697400e+10,USD,0.106066,16.193371,56.928894,0.176000,0.174000,0.197638
3,AAPL,Apple Inc.,United States,Consumer Electronics,2025,4.161610e+11,USD,0.311796,26.915064,46.905164,7.490000,7.460000,1.519130
4,AAPL,Apple Inc.,United States,Consumer Electronics,2024,3.910350e+11,USD,0.256825,23.971256,46.206350,6.110000,6.080000,1.645935
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174,PUK,Prudential plc,Hong Kong,Insurance - Life,2023,1.914800e+10,USD,0.009772,8.883434,NaN,1.242000,1.238000,0.095438
1175,PUK,Prudential plc,Hong Kong,Insurance - Life,2021,2.650000e+10,USD,-0.010848,-7.705660,NaN,-1.554000,-1.554000,-0.107837
1176,CRWV,"CoreWeave, Inc.",United States,Software - Infrastructure,2025,3.559096e+09,USD,NaN,NaN,NaN,NaN,NaN,NaN
1177,CRWV,"CoreWeave, Inc.",United States,Software - Infrastructure,2024,1.915426e+09,USD,-0.048420,-45.078640,74.243328,-2.330000,-2.330000,2.087650


In [135]:
comapnies_number=len(df["ticker"].unique())
print(comapnies_number)

comapnies_number=df["country"].unique()
print(comapnies_number)

comapnies_number= df["revenue_unit"].unique()
print(comapnies_number)

393
['United States' 'Netherlands' 'Hong Kong' 'China' 'United Kingdom'
 'Germany' 'Japan' 'Switzerland' 'Denmark' 'Canada' 'India' 'Ireland'
 'Spain' 'France' 'Australia' 'Luxembourg' 'Belgium' 'Uruguay' 'Brazil'
 'Singapore' 'Mexico' 'Italy' 'Norway' 'Indonesia']
['USD' 'EUR' 'CNY' 'JPY' 'DKK' 'CAD' 'INR' 'GBP' 'BRL' 'MXN']


In [91]:
df.to_csv('financials.csv', index=False)

In [136]:
df.head()

,ticker,company_name,country,industry,year,revenue,revenue_unit,ROA,net_profit_margin,gross_profit_margin,basic_eps,diluted_eps,roe
0,NVDA,NVIDIA Corporation,United States,Semiconductors,2025,1.304970e+11,USD,0.653041,55.848027,74.988697,2.970,2.940,0.918729
1,NVDA,NVIDIA Corporation,United States,Semiconductors,2024,6.092200e+10,USD,0.452775,48.849348,72.717573,1.210,1.190,0.692447
2,NVDA,NVIDIA Corporation,United States,Semiconductors,2023,2.697400e+10,USD,0.106066,16.193371,56.928894,0.176,0.174,0.197638
3,AAPL,Apple Inc.,United States,Consumer Electronics,2025,4.161610e+11,USD,0.311796,26.915064,46.905164,7.490,7.460,1.519130
4,AAPL,Apple Inc.,United States,Consumer Electronics,2024,3.910350e+11,USD,0.256825,23.971256,46.206350,6.110,6.080,1.645935


In [137]:
df.isna().sum()

ticker                   0
company_name            18
country                  0
industry                 0
year                     0
revenue                  0
revenue_unit             0
ROA                    302
net_profit_margin      301
gross_profit_margin    419
basic_eps              307
diluted_eps            307
roe                    308
dtype: int64

In [138]:
df = df.dropna(subset=["company_name"]).copy()

In [95]:
df.isna().sum()

ticker                   0
company_name             0
country                  0
industry                 0
year                     0
revenue                  0
revenue_unit             0
ROA                    297
net_profit_margin      297
gross_profit_margin    415
basic_eps              303
diluted_eps            303
roe                    303
dtype: int64

In [139]:
def clean_numeric(value):
    if pd.isna(value):
        return None
    v = str(value)
    v = v.replace(",", "")
    v = re.sub(r"[^0-9eE\.\-]", "", v)
    try:
        return float(v)
    except:
        return None


In [ ]:
import re
df["revenue"] = df["revenue"].apply(clean_numeric)

In [ ]:
def human_format(value):
    if pd.isna(value):
        return None
    value = float(value)
    if value >= 1e12:
        return f"{value/1e12:.2f} T"
    elif value >= 1e9:
        return f"{value/1e9:.2f} B"
    elif value >= 1e6:
        return f"{value/1e6:.2f} M"
    else:
        return f"{value:,.0f}"

In [142]:
df["revenue"] = df["revenue"].apply(human_format)

In [ ]:
df = df.sort_values(["ticker", "year"])
df[['ROA','net_profit_margin','gross_profit_margin','basic_eps','diluted_eps','roe']] = (
    df.groupby("ticker")[
        ['ROA','net_profit_margin','gross_profit_margin','basic_eps','diluted_eps','roe']
    ].ffill()
)

In [145]:
import pickle

with open('dataframe.pkl', 'wb') as f:
    pickle.dump(df, f)